# Pipelines_CrossValidation

In [261]:
#importamos las librerias necesarias

from sklearn.ensemble import RandomForestRegressor
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
import pandas as pd
from pathlib import Path
import os
import numpy as np
from matplotlib import pyplot
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score

In [262]:
#Cargamos la ruta
root=Path(".").resolve().parent
filename="melb_data.csv"
pathfile=os.path.join(root,'data','raw',filename)
#Cargamos la data
df=pd.read_csv(pathfile)
df.head()

,Suburb,Address,Rooms,Type,Price,Method,SellerG,Date,Distance,Postcode,...,Bathroom,Car,Landsize,BuildingArea,YearBuilt,CouncilArea,Lattitude,Longtitude,Regionname,Propertycount
0,Abbotsford,85 Turner St,2,h,1480000.0,S,Biggin,3/12/2016,2.5,3067.0,...,1.0,1.0,202.0,NaN,NaN,Yarra,-37.7996,144.9984,Northern Metropolitan,4019.0
1,Abbotsford,25 Bloomburg St,2,h,1035000.0,S,Biggin,4/02/2016,2.5,3067.0,...,1.0,0.0,156.0,79.0,1900.0,Yarra,-37.8079,144.9934,Northern Metropolitan,4019.0
2,Abbotsford,5 Charles St,3,h,1465000.0,SP,Biggin,4/03/2017,2.5,3067.0,...,2.0,0.0,134.0,150.0,1900.0,Yarra,-37.8093,144.9944,Northern Metropolitan,4019.0
3,Abbotsford,40 Federation La,3,h,850000.0,PI,Biggin,4/03/2017,2.5,3067.0,...,2.0,1.0,94.0,NaN,NaN,Yarra,-37.7969,144.9969,Northern Metropolitan,4019.0
4,Abbotsford,55a Park St,4,h,1600000.0,VB,Nelson,4/06/2016,2.5,3067.0,...,1.0,2.0,120.0,142.0,2014.0,Yarra,-37.8072,144.9941,Northern Metropolitan,4019.0


In [263]:
X=df.drop('Price',axis=1)
y=df['Price']

print("X_Columns: ",X.columns)
print("y_columns: ",y)


X_Columns:  Index(['Suburb', 'Address', 'Rooms', 'Type', 'Method', 'SellerG', 'Date',
       'Distance', 'Postcode', 'Bedroom2', 'Bathroom', 'Car', 'Landsize',
       'BuildingArea', 'YearBuilt', 'CouncilArea', 'Lattitude', 'Longtitude',
       'Regionname', 'Propertycount'],
      dtype='object')
y_columns:  0        1480000.0
1        1035000.0
2        1465000.0
3         850000.0
4        1600000.0
           ...    
13575    1245000.0
13576    1031000.0
13577    1170000.0
13578    2500000.0
13579    1285000.0
Name: Price, Length: 13580, dtype: float64


In [264]:
#Identificamos las variables num y categoricas las cuales nos tenga un cardinalidal menor a 10
cols_num=[cname for cname in X.columns if X[cname].dtypes in ['int64','float64']]
cols_cat=[cname for cname in X.columns if X[cname].dtypes == 'object' and len(X[cname].unique())<10]
print("Columns Num: ",cols_num)
print("Columns Cat: ",cols_cat)

Columns Num:  ['Rooms', 'Distance', 'Postcode', 'Bedroom2', 'Bathroom', 'Car', 'Landsize', 'BuildingArea', 'YearBuilt', 'Lattitude', 'Longtitude', 'Propertycount']
Columns Cat:  ['Type', 'Method', 'Regionname']


In [265]:

for col in cols_cat:
    if X[col].isnull().sum()>0:
        X[col].fillna(method='bfill',axis=0,inplace=True,)
        
F=pd.get_dummies(X[cols_cat],prefix=col,dtype='int')
X_new=pd.concat([X[cols_num],F],axis=1)
X_new.head()


,Rooms,Distance,Postcode,Bedroom2,Bathroom,Car,Landsize,BuildingArea,YearBuilt,Lattitude,...,Regionname_SP,Regionname_VB,Regionname_Eastern Metropolitan,Regionname_Eastern Victoria,Regionname_Northern Metropolitan,Regionname_Northern Victoria,Regionname_South-Eastern Metropolitan,Regionname_Southern Metropolitan,Regionname_Western Metropolitan,Regionname_Western Victoria
0,2,2.5,3067.0,2.0,1.0,1.0,202.0,NaN,NaN,-37.7996,...,0,0,0,0,1,0,0,0,0,0
1,2,2.5,3067.0,2.0,1.0,0.0,156.0,79.0,1900.0,-37.8079,...,0,0,0,0,1,0,0,0,0,0
2,3,2.5,3067.0,3.0,2.0,0.0,134.0,150.0,1900.0,-37.8093,...,1,0,0,0,1,0,0,0,0,0
3,3,2.5,3067.0,3.0,2.0,1.0,94.0,NaN,NaN,-37.7969,...,0,0,0,0,1,0,0,0,0,0
4,4,2.5,3067.0,3.0,1.0,2.0,120.0,142.0,2014.0,-37.8072,...,0,1,0,0,1,0,0,0,0,0


In [266]:
# asignamos el set de datos train y test

X_train, X_valid, y_train, y_valid = train_test_split(X_new,y,random_state=13, test_size=0.3,train_size=0.7)
col_nulls_num=[]
#Aplicamos imputacion a los valores de X y y
imp=SimpleImputer()
for col in cols_num:
    if X_train[col].isnull().sum()>0:
        X_train[col+'_missing']=X_train[col].isnull()
        X_valid[col+'_missing']=X_valid[col].isnull()
        col_nulls_num.append(col)
print(col_nulls_num)
X_columns=X_train.columns


['Car', 'BuildingArea', 'YearBuilt']


In [268]:
imp_X_train=pd.DataFrame(imp.fit_transform(X_train))
imp_X_valid=pd.DataFrame(imp.transform(X_valid))
imp_X_train.columns=X_columns
imp_X_valid.columns=X_columns
model=RandomForestRegressor(random_state=13)
model.fit(imp_X_train,y_train)
model.score(imp_X_train,y_train)

0.9701208309096643

In [270]:
mypipeline=Pipeline(steps=[('preprocessor',SimpleImputer()),('model',RandomForestRegressor(n_estimators=100,random_state=13))])
scores=-1*cross_val_score(mypipeline,X[cols_num],y,cv=5,scoring='neg_mean_absolute_error')
print(scores)
print(scores.mean())
mypipeline.fit(X[cols_num],y)
mypipeline.score(X[cols_num],y)

[211562.18679588 201798.62530209 191045.80863481 162088.18959306
 164391.85893348]
186177.33385186546


0.9705932892434286